#Resnet with Bayesian Output Layer

## Loading Training and Testing Sets

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython import display
import os
from PIL import Image
from torch.utils.data.dataset import Dataset

In [ ]:
#load trainingset

from google.colab import drive
drive.mount('/content/gdrive')
PATH = F"/content/gdrive/My Drive/Research/bnn/resnet/cifar10/testset.pth"
testset = torch.load(PATH,  map_location='cpu')
print(len(testset))

PATH = F"/content/gdrive/My Drive/Research/bnn/resnet/cifar10/trainset.pth"
trainset = torch.load(PATH,  map_location='cpu')
print(len(trainset))

Mounted at /content/gdrive
10000
50000


## Loading Bayesian Neural Network

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython import display
import os
from PIL import Image
from torch.utils.data.dataset import Dataset

!pip install pillow
#from scipy.misc import imread

%matplotlib inline

class NN(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        output = self.fc1(x)
        output = F.relu(output)
        output = self.out(output)
        return output

Initializing Bayesian Neural Network Architecture

In [ ]:
!pip3 install pyro-ppl==1.5.0
import pyro
from pyro.distributions import Normal, Categorical
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

log_softmax = nn.LogSoftmax(dim=1)

     |████████████████████████████████| 604 kB 5.0 MB/s 


In [ ]:
bnet = NN(512*4*4,1024,10)

In [ ]:
def model(x_data, y_data):
    
    fc1w_prior = Normal(loc=torch.zeros_like(bnet.fc1.weight), scale=torch.ones_like(bnet.fc1.weight))
    fc1b_prior = Normal(loc=torch.zeros_like(bnet.fc1.bias), scale=torch.ones_like(bnet.fc1.bias))
    
    outw_prior = Normal(loc=torch.zeros_like(bnet.out.weight), scale=torch.ones_like(bnet.out.weight))
    outb_prior = Normal(loc=torch.zeros_like(bnet.out.bias), scale=torch.ones_like(bnet.out.bias))
    
    priors = {'fc1.weight': fc1w_prior, 'fc1.bias': fc1b_prior,  'out.weight': outw_prior, 'out.bias': outb_prior}
    # lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module", bnet, priors)
    # sample a regressor (which also samples w and b)
    lifted_reg_model = lifted_module()
    
    lhat = log_softmax(lifted_reg_model(x_data))
    
    pyro.sample("obs", Categorical(logits=lhat), obs=y_data)

In [ ]:
softplus = torch.nn.Softplus()

def guide(x_data, y_data):
    
    # First layer weight distribution priors
    fc1w_mu = torch.randn_like(bnet.fc1.weight)
    fc1w_sigma = torch.randn_like(bnet.fc1.weight)
    fc1w_mu_param = pyro.param("fc1w_mu", fc1w_mu)
    fc1w_sigma_param = softplus(pyro.param("fc1w_sigma", fc1w_sigma))
    fc1w_prior = Normal(loc=fc1w_mu_param, scale=fc1w_sigma_param)
    # First layer bias distribution priors
    fc1b_mu = torch.randn_like(bnet.fc1.bias)
    fc1b_sigma = torch.randn_like(bnet.fc1.bias)
    fc1b_mu_param = pyro.param("fc1b_mu", fc1b_mu)
    fc1b_sigma_param = softplus(pyro.param("fc1b_sigma", fc1b_sigma))
    fc1b_prior = Normal(loc=fc1b_mu_param, scale=fc1b_sigma_param)
    # Output layer weight distribution priors
    outw_mu = torch.randn_like(bnet.out.weight)
    outw_sigma = torch.randn_like(bnet.out.weight)
    outw_mu_param = pyro.param("outw_mu", outw_mu)
    outw_sigma_param = softplus(pyro.param("outw_sigma", outw_sigma))
    outw_prior = Normal(loc=outw_mu_param, scale=outw_sigma_param).independent(1)
    # Output layer bias distribution priors
    outb_mu = torch.randn_like(bnet.out.bias)
    outb_sigma = torch.randn_like(bnet.out.bias)
    outb_mu_param = pyro.param("outb_mu", outb_mu)
    outb_sigma_param = softplus(pyro.param("outb_sigma", outb_sigma))
    outb_prior = Normal(loc=outb_mu_param, scale=outb_sigma_param)
    priors = {'fc1.weight': fc1w_prior, 'fc1.bias': fc1b_prior, 'out.weight': outw_prior, 'out.bias': outb_prior}
    
    lifted_module = pyro.random_module("module", bnet, priors)
    
    return lifted_module()

In [ ]:
optim = Adam({"lr": 0.01})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

Train Bayesian Neural Network 

In [ ]:
#load dataset
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

#train network
num_iterations = 3
loss = 0

for j in range(num_iterations):
    loss = 0
    for batch_id, data in enumerate(trainloader):
        # calculate the loss and take a gradient step
        #print(data[0].size())
        #print(data[1].size())
        #print(data[1])
        #data_new = data[0].view(-1,512*4*4)
        #print(data_new.size())
        loss += svi.step(data[0].view(-1,512*4*4), data[1])
    normalizer_train = len(trainloader.dataset) 
    print(normalizer_train)
    total_epoch_loss_train = loss / normalizer_train
    print("Epoch ", j, " Loss ", total_epoch_loss_train)

/usr/local/lib/python3.7/dist-packages/pyro/primitives.py:406: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  "modules from `torch.nn.Module` instances.", FutureWarning)


50000
Epoch  0  Loss  17696.345529101865
50000
Epoch  1  Loss  771.588113425931
50000
Epoch  2  Loss  370.9168674097791


Testing the State Dict

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in bnet.state_dict():
    print(param_tensor, "\t", bnet.state_dict()[param_tensor].size())

Model's state_dict:
fc1.weight 	 torch.Size([1024, 8192])
fc1.bias 	 torch.Size([1024])
out.weight 	 torch.Size([10, 1024])
out.bias 	 torch.Size([10])


Saving the model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


# good example: https://github.com/pyro-ppl/pyro/blob/dev/examples/dmm.py

# saves the model and optimizer states to disk
def save_checkpoint():
    PATH = '/content/gdrive/My Drive/Research/bnn/resnet/cifar10/bnet_state_dict.pth'
    torch.save(bnet.state_dict(), PATH)
    PATH = '/content/gdrive/My Drive/Research/bnn/resnet/cifar10/bnet_optim.pth'
    optim.save(PATH)

# loads the model and optimizer states from disk
def load_checkpoint():
    PATH = '/content/gdrive/My Drive/Research/bnn/resnet/cifar10/bnet_state_dict.pth'
    bnet.load_state_dict(torch.load(PATH))
    PATH = '/content/gdrive/My Drive/Research/bnn/resnet/cifar10/bnet_optim.pth'
    optim.load(PATH)

save_checkpoint()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Loading the model

## Accuracy Tests

BNN Test When Network Is *Forced* To Predict

In [ ]:
num_samples = 10
def predict(x):
    sampled_models = [guide(None, None) for _ in range(num_samples)]
    yhats = [model(x).data for model in sampled_models]
    mean = torch.mean(torch.stack(yhats), 0)
    return np.argmax(mean.numpy(), axis=1)


In [ ]:

#load dataset
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True, num_workers=2)


print('Prediction when network is forced to predict')
correct = 0
total = 0
for j, data in enumerate(testloader):
    images, labels = data
    predicted = predict(images.view(-1,512*4*4))
    total += labels.size(0)
    predicted = torch.Tensor(predicted)
    correct += (predicted.eq(labels).sum().item())
print('Accuracy of the BNN network on the 10000 test images: %d %%' % (100 * correct / total))


Prediction when network is forced to predict


/usr/local/lib/python3.7/dist-packages/pyro/primitives.py:406: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  "modules from `torch.nn.Module` instances.", FutureWarning)


Accuracy of the BNN network on the 10000 test images: 94 %


In [ ]:
#to optimize for space
del trainset
del trainloader
del testset
del testloader

BNN Test When Network Can Be Undecided

In [ ]:
#BNN TEST WHEN IT CAN REFUSE

num_samples = 100
def give_uncertainities(x):
      sampled_models = [guide(None, None) for _ in range(num_samples)]
      yhats = [F.log_softmax(model(x.view(-1,512*4*4)).data, 1).detach().numpy() for model in sampled_models]
      return np.asarray(yhats)
      #mean = torch.mean(torch.stack(yhats), 0)
      #return np.argmax(mean, axis=1)

def bnn_test(test_loader, plot=True):
  predicted_for_images = 0
  correct_predictions=0 
  total = 0
  for j, data in enumerate(test_loader):
      images, labels = data
      y = give_uncertainities(images)        

      for i in range(len(labels)):
          all_digits_prob = []
          highted_something = False
          for j in range(10):
              highlight=False
              histo = []
              histo_exp = []
              for z in range(y.shape[0]):
                  histo.append(y[z][i][j])
                  histo_exp.append(np.exp(y[z][i][j]))
              prob = np.percentile(histo_exp, 10) #sampling median probability, sampling 10th percentile to be 50%
              if(prob>0.5): #select if network thinks this sample is 50% chance of this being a label
                  highlight = True #possibly an answer
              all_digits_prob.append(prob)
          
              if(highlight):
                  highted_something = True
      
          predicted = np.argmax(all_digits_prob)
      
          if(highted_something):
              predicted_for_images+=1
              if(labels[i].item()==predicted):
                  correct_predictions +=1.0

      total+= len(labels)
      
  if(plot):
          print("Summary")
          print("Total images: ", total, "10,000")
          print("Predicted for: ", predicted_for_images)
          print("Correct Images ", correct_predictions)
          print("Accuracy when predicted: ",correct_predictions/predicted_for_images)
          good = ((total - predicted_for_images) + correct_predictions)*100 /total
          print("Good Score", good)

  return total, predicted_for_images, int(correct_predictions)

In [ ]:
import pandas as pd
dataframe = pd.DataFrame(columns=['Sigmas','Total','Predicted','Correct'])

sigmas = [0.05, 0.15, 0.25, 0.35, 0.45]
sigma_list = []
total = []
predicted = []
correct1 = []

for sigma in sigmas:
  correct_path = f"/content/gdrive/My Drive/Research/bnn/resnet/cifar10/testset_{sigma}_c.pth"
  correct = torch.load(correct_path, map_location='cpu')
  testloader_c = torch.utils.data.DataLoader(correct, batch_size=128, shuffle=False, num_workers=2)
  tot, pred, cor = bnn_test(testloader_c)
  sigma_list.append(sigma)
  total.append(tot)
  predicted.append(pred)
  correct1.append(cor)
  print("Tested BNN on SNN correct data for sigma = ",sigma)
  del correct
  del testloader


for sigma in sigmas:
  incorrect_path = f"/content/gdrive/My Drive/Research/bnn/resnet/cifar10/testset_{sigma}_f.pth"
  incorrect = torch.load(incorrect_path, map_location='cpu')
  testloader_f = torch.utils.data.DataLoader(incorrect, batch_size=128, shuffle=False, num_workers=2)
  tot, pred, cor = bnn_test(testloader_f)
  sigma_list.append(sigma)
  total.append(tot)
  predicted.append(pred)
  correct1.append(cor)
  print("Tested BNN on VGG incorrect data for sigma = ",sigma)

dataframe['Sigmas'] = sigma_list
dataframe['Total'] = total
dataframe['Predicted'] = predicted
dataframe['Correct'] = correct1

dataframe_path = F"/content/gdrive/My Drive/Research/bnn/restnet/cifar10/cifar10_noise_0.csv"

dataframe.to_csv(dataframe_path)




/usr/local/lib/python3.7/dist-packages/pyro/primitives.py:406: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  "modules from `torch.nn.Module` instances.", FutureWarning)


Summary
Total images:  9490 10,000
Predicted for:  0
Correct Images  0


ZeroDivisionError: ignored

In [ ]:
import pandas as pd
dataframe = pd.DataFrame(columns=['Epsilon','Total','Predicted','Correct'])

epsilon_list  = [0,1,2,3,4,5,6,7,8,9]
epsilon_strength = []
total = []
predicted = []
correct1 = []


for epsilon in epsilon_list:
  correct_path = f"/content/gdrive/My Drive/Research/bnn/resnet/cifar10/attack_testsets/attack_testset_{epsilon}_c.pth"
  correct = torch.load(correct_path, map_location='cpu')
  testloader_c = torch.utils.data.DataLoader(correct, batch_size=128, shuffle=False, num_workers=2)
  tot, pred, cor = bnn_test(testloader_c)
  epsilon_strength.append(epsilon)
  total.append(tot)
  predicted.append(pred)
  correct1.append(cor)
  print("Tested BNN on SNN correct data for epsilon = ",epsilon)
  del correct
  del testloader_c



for epsilon in epsilon_list:
  incorrect_path = f"/content/gdrive/My Drive/Research/bnn/resnet/cifar10/attack_testsets/attack_testset_{epsilon}_c.pth"
  incorrect = torch.load(incorrect_path, map_location='cpu')
  testloader_f = torch.utils.data.DataLoader(incorrect, batch_size=128, shuffle=False, num_workers=2)
  tot, pred, cor = bnn_test(testloader_f)
  epsilon_strength.append(epsilon)
  total.append(tot)
  predicted.append(pred)
  correct1.append(cor)
  print("Tested BNN on VGG incorrect data for epsilon = ",epsilon)
  del incorrect
  del testloader_f

dataframe['Epsilon'] = epsilon_strength
dataframe['Total'] = total
dataframe['Predicted'] = predicted
dataframe['Correct'] = correct1

dataframe_path = "/content/gdrive/My Drive/Research/bnn/resnet/cifar10/adversarial_data.pth"
dataframe.to_csv(dataframe_path)


/usr/local/lib/python3.7/dist-packages/pyro/primitives.py:406: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  "modules from `torch.nn.Module` instances.", FutureWarning)


Summary
Total images:  9528 10,000
Predicted for:  8955
Correct Images  8947.0
Accuracy when predicted:  0.999106644332775
Good Score 99.91603694374476
Tested BNN on SNN correct data for epsilon =  0
Summary
Total images:  8313 10,000
Predicted for:  6922
Correct Images  6896.0
Accuracy when predicted:  0.9962438601560243
Good Score 99.68723685793336
Tested BNN on SNN correct data for epsilon =  1
Summary
Total images:  8036 10,000
Predicted for:  6633
Correct Images  6606.0
Accuracy when predicted:  0.9959294436906377
Good Score 99.66401194624191
Tested BNN on SNN correct data for epsilon =  2
